In [19]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:85%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

<span style="color:red; font-size:35px;"><strong> ch2. LLM활용의 기본 개념(ollama)</strong></span>
# 1. LLM을 활용하여 답변 생성하기 
## 1) Ollama 이용한 로컬 LLM 이용
 성능은 GPT, Claude 같은 모델보다 떨어지나, 개념 설명을 위해 open source 모델 사용
 
### ollama.com 다운로드 -> 설치 -> 모델 pull
- ollama pull deepseek-r1:1.5b (window키 + R → Powershell 검색)

In [4]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:1.5b")
result = llm.invoke("What is the capital of Korea?")
result # 추론 모델 <think>~<think>

AIMessage(content="<think>\n\n</think>\n\nKorea, officially known as the Democratic People's Republic of Korea ( temporarily referred to as the Republic of Korea in some countries), has its capital city at Seoul.", additional_kwargs={}, response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-06-25T02:09:38.5588129Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3466996700, 'load_duration': 26997300, 'prompt_eval_count': 10, 'prompt_eval_duration': 77547700, 'eval_count': 38, 'eval_duration': 3362451700, 'model_name': 'deepseek-r1:1.5b'}, id='run--0bc069f3-0e7d-404f-a543-a153d91f25c7-0', usage_metadata={'input_tokens': 10, 'output_tokens': 38, 'total_tokens': 48})

### ollama.com 다운로드 → 설치→ 모델 pull
- ollama pull llama3.2:1b (window키+R → powershell 검색)
- llama : 공식적으로 한글지원 안 됨 (llama3.1 405b한글지원 가능 -> llama3.3 70b)
- exaone : 공식적으로 한글지원 

In [8]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("what is the capital of Korea?")
result.content

"The capital of South Korea is Seoul. However, it's worth noting that the Demilitarized Zone (DMZ) surrounding the two Koreas also claims Pyongyang as its capital. The North Korean government still uses Pyongyang as its capital, while South Korea recognizes Seoul as its de facto capital. This is because the DMZ is not controlled by either side and serves as a buffer zone between the two countries."

In [12]:
result = llm.invoke("한국 수도는 어디야?")
result.content

'한가지 önemliething은 한국 수도입니다. 한국의 수도는 Seoul로 정기적으로 운영됩니다.'

## 2) openai 활용
- pip install lanchain-openai

In [13]:
from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini")
# result = llm.invoke("what is the capital of Korea?")
# result
# 에러사유 : OPENAI_API_KEY 환경변수 부재

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [16]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
# os.getenv('OPENAI_API_KEY')

True

In [ ]:
# 코랩에서 OPENAI_API_KEY 읽어오기(코랩에서는 .env를 못 씀)
# from google.colab import userdata
# userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=#'gpt-4.1-nano', 
                 # openai_api_key=os.getenv('OPENAI_API_KEY'))
llm.invoke("what is capital of Korea? Answer me in Korea")

In [ ]:
# 모든 모델의 키가 OPEN_API_KEY는 아님
# Claude -> Anthropic
# Azure, upstage, Bedrock : 에러 메세지 참조하여 환경변수 생성

In [24]:
# from langchain_openai import AzureOpenAI
# llm = AzureOpenAI(model='')
# llm.invoke("what is the capital of Korea?")
# 에러 :OPENAI_API_VERSION 환경변수가 필요하다는 메세지

ValidationError: 1 validation error for AzureOpenAI
  Value error, Must provide either the `api_version` argument or the `OPENAI_API_VERSION` environment variable [type=value_error, input_value={'model': '', 'model_kwargs': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [29]:
from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model='claude-3-5-sonnet-20240620')
# llm.invoke('What is the capital of Korea?')
# 에러 메세지를 봐도 환경변수 이름을 알 수 없음 → ChatAnthropic 검색 후, Langchain docs 
# langchain docs에서 명시한 ANTHROPIC_API_KEY이름의 환경변수 설정
# https://python.langchain.com/docs/integrations/chat/anthropic/

# 2. 렝체인 스타일로 프롬프트 작성하기
- 프롬프트:llm 호출시 쓰는 질문

In [73]:
from langchain_ollama import ChatOllama
llm=ChatOllama(model="llama3.2:1b")
# llm.invoke(0)
# 프로프트 타입 : 스트링, PromptValue, BaseMessage 리스트

## 1)기본 프롬프트 템플릿
- PromptTemplate을 사용하여 변수가 포함된 템플릿 작성

In [74]:
from langchain_core.prompts import PromptTemplate
llm= ChatOllama(model="llama3.2:1b")
prompt_template = PromptTemplate(
    template = "What is the capital of {country}", # {} 안의 값을 새로운 값으로 대입 가능 
    input_variables = ["country"]
)
prompt = prompt_template.invoke({"country":"Korea"})
print(prompt)
# llm.invoke(prompt_template.invoke({"country":"Korea"})) # country에 korea 

text='What is the capital of Korea'


## 2) 메세지 기반 프롬프트 작성
- BaseMessage 리스트
- BaseMessage 상속 받은 클래스 : AIMessage, HummanMessage, SystemMess, ToolMessage

In [75]:
# BaseMessage List로 하면 렝체인화X, ChatPromptTemplateX 
# 튜플리스트로 변경
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
llm = ChatOllama(model="llama3.2:1b")
message_list = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="What is the capital of Italy?"),
    AIMessage(content="The capital of Italy is Roma"),
    HumanMessage(content="What is the capital of Korea?"),
    AIMessage(content="The capital of Italy is Seoul"),
    HumanMessage(content="What is the capital of France?")
] # 단순한 리스트라서 템플릿을 더 거쳐야 함
from langchain_core.prompts import ChatPromptTemplate
chatpromptTemplate = ChatPromptTemplate.from_messages(message_list) # message를 프롬프트화 시킴
prompt = chatpromptTemplate.invoke({"country":"Korea"})
print(prompt)

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Italy is Roma', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Korea?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Italy is Seoul', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]


## 3)ChatPromptTemplate 사용
- BaseMessage 리스트 → 튜플리스트 


In [76]:
# 위의 BaseMessage를 수정
chatpromptTemplate = ChatPromptTemplate.from_messages([
    ("system","당신은 대한민국 전문 도우미에요!"),
    ("human","{country}의 수도가 어디에요?"),
    
])
country=input("어느 나라 수도가 궁금하세요?")
prompt = chatpromptTemplate.invoke({"country":country})
print("프롬프트 :",prompt)
result = llm.invoke(prompt)
result.content

어느 나라 수도가 궁금하세요?
프롬프트 : messages=[SystemMessage(content='당신은 대한민국 전문 도우미에요!', additional_kwargs={}, response_metadata={}), HumanMessage(content='의 수도가 어디에요?', additional_kwargs={}, response_metadata={})]


'대한민국의 의 수도는 Seoul입니다. 그것은 주로 경인도에서 이끄는 곳이지만, 북부와 남부 cũng 일부 지역을 지배하고 있습니다.'

# 3. 답변 형식을 컨트롤하기
- invoke 실행결과는 AIMessage() → String이나 json,객체 : outputParser이용

## 1) 문자열 출력 파서 이용
- strOutputParser를 사용하여 LLM출력(AIMessage)을 단순 문자열로 변환

In [77]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}.  Return the name of the city only",
    input_variables = ["country"]
)
# 프롬프트 템플릿에 값 주입
prompt = prompt_template.invoke({"country":"korea"})
print("프롬프트 :",prompt)
result = llm.invoke(prompt)
print("llm 결과 :",type(result), result)
# 문자열 출력 파서를 이용하여 llm응답을 단순 문자열 반환
output_parser = StrOutputParser()
print("파서 결과 :",output_parser.invoke(result))

프롬프트 : text='What is the capital of korea.  Return the name of the city only'
llm 결과 : <class 'langchain_core.messages.ai.AIMessage'> content='Seoul' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-25T07:05:16.5330802Z', 'done': True, 'done_reason': 'stop', 'total_duration': 724516400, 'load_duration': 26999600, 'prompt_eval_count': 41, 'prompt_eval_duration': 576951800, 'eval_count': 3, 'eval_duration': 119808100, 'model_name': 'llama3.2:1b'} id='run--39599303-5a03-4688-94d1-2ce84afbacc5-0' usage_metadata={'input_tokens': 41, 'output_tokens': 3, 'total_tokens': 44}
파서 결과 : Seoul


In [78]:
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

In [81]:
# PromptTemplate(변수설정) => ChatPromptTemplate(변수설정, system과 모법답안 지정)
llm = ChatOllama(model="llama3.2:1b")

chat_prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant with expertise in South Korea."),
    ("human", "What is the capital of {country}? Return the name if the city only.")
])

output_parser = StrOutputParser()

output_parser.invoke(llm.invoke(chat_prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) JSON 출력 파서 이용
- jsom()으로 응답하기를 원하지만, 우선 어떤 형식으로 반환되는지 확인
- {"name":"홍","age":"22"(josn)/{"name":"홍","age"

In [82]:
from langchain_core.output_parsers import JsonOutputParser
count_detail_prompr = PromptTemplate(
    template = """Give Following information about {country}
    - Capital
    - Popultion
    - Languate
    - Currency
    return it is JSON format and return the JSON dictionart only""",
    input_variables=["country"]
)
prompt = count_detail_prompr.invoke({"country":"Korea"})
print(type(prompt),prompt)
# Json output 파서
output_parser = JsonOutputParser()
ai_message = llm.invoke(prompt)
print(type(ai_message), ai_message)
json_result = output_parser.invoke(ai_message)
print(type(json_result),json_result)

<class 'langchain_core.prompt_values.StringPromptValue'> text='Give Following information about Korea\n    - Capital\n    - Popultion\n    - Languate\n    - Currency\n    return it is JSON format and return the JSON dictionart only'
<class 'langchain_core.messages.ai.AIMessage'> content='```\n{\n    "capital": "Seoul",\n    "population": 51.8,\n    "language": "Korean (based on Hangul)",\n    "currency": "South Korean Won"\n}\n```' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-25T07:06:08.9298263Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4060633700, 'load_duration': 30070200, 'prompt_eval_count': 65, 'prompt_eval_duration': 1021683500, 'eval_count': 45, 'eval_duration': 3008112800, 'model_name': 'llama3.2:1b'} id='run--76248fb7-609b-4bb5-8733-fcb4a36bb324-0' usage_metadata={'input_tokens': 65, 'output_tokens': 45, 'total_tokens': 110}
<class 'dict'> {'capital': 'Seoul', 'population': 51.8, 'language': 'Korean (based on Hangul)', 

In [104]:
count_detail_prompr = PromptTemplate(
    template = """Give Following information about {country}
    - Capital
    - Popultion
    - Language
    - Currency
    return it is JSON format and return the JSON dictionart only""",
    input_variables=["country"]
)
output_parser = JsonOutputParser()
info = output_parser.invoke(llm.invoke(count_detail_prompr.invoke({"country":"Korea"})))
info

{'capital': 'Seoul',
 'population': 51.8,
 'language': 'Korean',
 'currency': 'Korean Won'}

In [85]:
type(info)

dict

## 3) 구조화된 출력 사용
- Pydantic 모델을 사용하여 LLM 출력을 구조화된 형식으로 받기(Jsonparser보다 훨씬 안정적)
- Pydantic : 데이터 유효성 검사, 설정관리를 간편하게 해주는 라이브러리

In [100]:
from pydantic import BaseModel,Field
class User:
    def __init__(self, id, name, is_active=True):
        self.id = id
        self.name = name
        self.is_active = is_active
#         if len(name)<=1:
#             throw Exception 
#     def __str__(self):
#         return self.name + " " + self.id
user = User(1,"홍길동")
print(user)

In [102]:
class User(BaseModel):
    # gt=0:id>0 / ge=0:id>=0 / lt=0:id<0 / le=0:id<=0
    id:int     = Field(gt=0, description="id") # 미니멈 값
    name:str   = Field(min_length=2, description="name")
    is_active:bool=Field(default=True,description="id활성화")
user = User(id="1", name="홍길동")
print(user)

id=1 name='홍길동' is_active=True


In [118]:
count_detail_prompr = PromptTemplate(
    template = """Give Following information about {country}
    - Capital
    - Popultion
    - Language
    - Currency
    return it is JSON format and return the JSON dictionart only""",
    input_variables=["country"]
)
class CountryDetail(BaseModel): # description : 더 정확한 출력 유도
    capital:str = Field(description="the capital of the country")
    population:int = Field(description="the population of the country")
    language:str = Field(description="the language of the country")
    currency:str = Field(description="the capital of the country")
# 출력 형식 파서 생성 (llm과 파싱이 같이 되는 기능)
structedllm = llm.with_structured_output(CountryDetail)

# 기존 방식 
# output_parser = JsonOutputParser()
# output_parser.invoke(llm.invoke(count_detail_prompr.invoke({"country":"Korea"})))

info = structedllm.invoke(count_detail_prompr.invoke({"country":"Korea"}))
info

CountryDetail(capital='Seoul', population=51000000, language='Korean', currency='Korean won')

In [113]:
type(info) # 타입은 객체로 받음

__main__.CountryDetail

In [119]:
print(info)
print(info.capital, info.population, info.language, info.currency)

capital='Seoul' population=51000000 language='Korean' currency='Korean won'
Seoul 51000000 Korean Korean won


In [120]:
print('info → JSON :', info.json()) # 곧 없어질 예정이라고 경고 발생 

info → JSON : {"capital":"Seoul","population":51000000,"language":"Korean","currency":"Korean won"}


C:\Users\Admin\AppData\Local\Temp\ipykernel_6456\2195821428.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print('info → JSON :', info.json())


In [123]:
print('info → JSON :', info.model_dump_json())

info → JSON : {"capital":"Seoul","population":51000000,"language":"Korean","currency":"Korean won"}


In [124]:
print('info → dict :', info.dict()) # 곧 없어질 예정이라고 경고 발생 

info → dict : {'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'Korean won'}


C:\Users\Admin\AppData\Local\Temp\ipykernel_6456\1042155660.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print('info → dict :', info.dict())


In [125]:
print('info → dict :', info.model_dump())

info → dict : {'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'Korean won'}


# 4. LCEL을 활용한 랭체인 생성하기
## 1) 문자열 출력 파서 사용
- invoke 

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b",
                 temperature=0) # 일관된 답변
# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}.  Return the name of the city only",
    input_variables = ["country"])

output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

invoke를 계속 사용하며 문장구성이 어지러움/ LCEL을 쓰면 invoke를 쓰지않고 연산자를 사용함

## 2) LCEL을 사용한 간단한 체인 구성
- 파이프연산자(|) 사용

In [4]:
# 프롬프트템플릿 → llm → 출력파서를 연결하는 체인 생성 
capital_chain = prompt_template | llm | output_parser
# 생성된 체인을 invoke
capital_chain.invoke({"country":"Korea"})

'Seoul'

In [5]:
type(capital_chain)

langchain_core.runnables.base.RunnableSequence

## 3) 복합 체인 구성
- 여러단계의 추론이 필요한 경우 (체인 연결)

In [7]:
# 나라 설명(입력) → 나라명(출력)
from langchain_core.prompts import PromptTemplate 
# 페르소나 지정X,ChatPromptTemplate는 페르소나 지정 
country_prompt = PromptTemplate(
    template = """Guess the name of the country based on the following information: 
    {information}
    Return the name of the the country only""",
    imput_variavles=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                                                       "This country is very famous for its wine"})))

'Italy.'

In [12]:
# 나라명 추측 체인 생성
country_chain = country_prompt | llm | output_parser
# type(country_chain)
country_chain.invoke({"information":"This country is very famous for its wine"})

'Italy.'

In [13]:
# 나라 설명(입력) → (나라명 → ) → 나라 수도(출력)
final_chain = country_chain | capital_chain 
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

In [15]:
final_chain = {"country" : country_chain} | capital_chain
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

In [16]:
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | {"country":country_chain} | capital_chain
final_chain.invoke("This country is very famous for its wine")

'Rome'

In [17]:
# 프롬프트 템플릿에 변수2 
# 나라설명 → 나라명
from langchain_core.prompts import PromptTemplate
country_prompt = PromptTemplate(
    template ="""Guess the name of the country in the {continent} based
    on the following information:{information}
    Return the name of the country only""",
    input_variables=["information","continent"])

country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information":"This country is very famous for its wine",
                      "continent":"Europe"})

'Italy'

In [22]:
final_chain = {"country" : country_chain} | capital_chain
final_chain.invoke({"information":"This country is very famous for its wine",
                    "continent":"Europe"})

'Rome'

In [ ]:
# Q. 나라명(입력) → (제일 유명한 음식 → ) → 음식의 레시피(출력)
